In [118]:
import pandas as pd
import numpy as np
import warnings

pd.set_option("mode.copy_on_write", True)

In [119]:
poverty_data=pd.read_csv("USDA_data/Poverty.csv")
# columns name : ['FIPS*', 'Name', 'RUC Code', 'All people in poverty (2021) Percent', 'Children ages 0-17 in poverty (2021) Percent']
poverty_data.rename(columns={'FIPS*': 'FIPS'}, inplace=True)
poverty_data["YR"]="2021" # add a column as Year
poverty_data.head(5)

,FIPS,Name,RUC Code,All people in poverty (2021) Percent,Children ages 0-17 in poverty (2021) Percent,YR
0,23000,Maine,NaN,11.2,13.8,2021
1,23001,Androscoggin,3.0,14.0,19.1,2021
2,23003,Aroostook,7.0,14.9,18.9,2021
3,23005,Cumberland,2.0,7.7,8.7,2021
4,23007,Franklin,8.0,11.7,15.7,2021


In [120]:
education_CompletingCollege=pd.read_csv("USDA_data/Education(CompletingCollege).csv")
education_CompletingCollege.rename(columns={'2023 Rural-urban Continuum Code*': 'RUC Code'}, inplace=True)

education_CompletingCollege_melt = education_CompletingCollege.melt(
    id_vars=['FIPS', 'Name','RUC Code'],
    var_name='YR',
    value_name='Completing College'
)

education_CompletingHighSchool_only=pd.read_csv("USDA_data/Education(CompletingHighSchool_only).csv")
education_CompletingHighSchool_only.rename(columns={'2023 Rural-urban Continuum Code*': 'RUC Code'}, inplace=True)

education_CompletingHighSchool_only_melt  = education_CompletingHighSchool_only.melt(
    id_vars=['FIPS', 'Name','RUC Code'],
    var_name='YR',
    value_name='Completing High School Only'
)


education_NotCompletingHighSchool=pd.read_csv("USDA_data/Education(NotCompletingHighSchool).csv")
education_NotCompletingHighSchool.rename(columns={'2023 Rural-urban Continuum Code*': 'RUC Code'}, inplace=True)
education_NotCompletingHighSchool_melt = education_NotCompletingHighSchool.melt(
    id_vars=['FIPS', 'Name','RUC Code'],
    var_name='YR',
    value_name='Not Completing High School'
)


In [135]:
education_CompletingHighSchool_only_2 = education_CompletingHighSchool_only_melt.drop(columns=['Name', 'RUC Code'])
education_NotCompletingHighSchool_2 = education_NotCompletingHighSchool_melt.drop(columns=['Name', 'RUC Code'])
# in case RUC Code and Name different expression

education_data = (
    education_CompletingCollege_melt
    .merge(education_CompletingHighSchool_only_2, on=['FIPS', 'YR'], how='inner')  # Inner join on FIPS and YR
    .merge(education_NotCompletingHighSchool_2, on=['FIPS', 'YR'], how='inner')   # Another inner join
    
)

education_data.head(5)

# check inner
# education_data = (
#     education_CompletingCollege_melt
#     .merge(education_CompletingHighSchool_only_2, on=['FIPS', 'YR'], how='outer', indicator=True)
#     .rename(columns={'_merge': '_merge_college_highschool'})  # Rename the first `_merge` column
#     .merge(education_NotCompletingHighSchool_2, on=['FIPS', 'YR'], how='outer', indicator=True)
#     .rename(columns={'_merge': '_merge_final'})  # Rename the final `_merge` column
# )

# # Analyze the final merge summary
# merge_summary = education_data['_merge_final'].value_counts()

# # Display the merge summary
# print("Summary of the final merge:")
# print(merge_summary)

,FIPS,Name,RUC Code,YR,Completing College,Completing High School Only,Not Completing High School
0,12000,Florida,NaN,1970,10.3%,30.7%,47.4%
1,12001,"Alachua, FL",2.0,1970,23.1%,24.5%,40.2%
2,12003,"Baker, FL",1.0,1970,3.6%,26.5%,65.1%
3,12005,"Bay, FL",3.0,1970,9.2%,31.5%,49.0%
4,12007,"Bradford, FL",6.0,1970,4.5%,25.7%,63.5%


In [122]:
unemployment_income=pd.read_csv("USDA_data/UnemploymentRate(%).csv")
unemployment_income.columns = unemployment_income.iloc[0]
unemployment_income = unemployment_income[2:].reset_index(drop=True)
unemployment_income = unemployment_income.iloc[:, :-1]
unemployment_income.rename(columns={'FIPS ': 'FIPS'}, inplace=True)
unemployment_income.head(5)


,FIPS,Name,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,Median Household Income (2021)
0,12001,"Alachua County, FL",5.4,4.7,4.3,3.9,3.3,3.0,5.7,3.8,2.9,"$56,006"
1,12003,"Baker County, FL",6.6,5.6,4.9,4.2,3.5,3.2,5.3,3.8,2.9,"$60,994"
2,12005,"Bay County, FL",6.5,5.5,4.8,4.2,4.1,3.8,6.3,4.0,2.8,"$60,106"
3,12007,"Bradford County, FL",5.8,4.9,4.3,3.7,3.4,3.3,6.3,4.3,3.3,"$54,911"
4,12009,"Brevard County, FL",7.2,6.0,5.2,4.4,3.6,3.3,7.0,4.1,2.9,"$65,280"


In [123]:
income_data = unemployment_income[['FIPS', 'Name', 'Median Household Income (2021)']]
income_data["YR"]="2021" # add a column as Year
income_data.head(5)

,FIPS,Name,Median Household Income (2021),YR
0,12001,"Alachua County, FL","$56,006",2021
1,12003,"Baker County, FL","$60,994",2021
2,12005,"Bay County, FL","$60,106",2021
3,12007,"Bradford County, FL","$54,911",2021
4,12009,"Brevard County, FL","$65,280",2021


In [124]:
unemployment_data=unemployment_income.iloc[:, :-1]
unemployment_data.head(5)
unemployment_data = unemployment_data.melt(
    id_vars=['FIPS', 'Name'],
    var_name='YR',
    value_name='Unemployment Rate (%)'
)
unemployment_data['YR'] = unemployment_data['YR'].astype(int)
unemployment_data.head(5)

,FIPS,Name,YR,Unemployment Rate (%)
0,12001,"Alachua County, FL",2014,5.4
1,12003,"Baker County, FL",2014,6.6
2,12005,"Bay County, FL",2014,6.5
3,12007,"Bradford County, FL",2014,5.8
4,12009,"Brevard County, FL",2014,7.2


# Merge all dataset

In [125]:
#check data type, prepare for merging

# print(education_data.dtypes)
# print(poverty_data.dtypes)
# print(income_data.dtypes)
# print(unemployment_data.dtypes)
education_data['YR'] = education_data['YR'].astype('str')
poverty_data['YR'] = poverty_data['YR'].astype('str')
income_data['FIPS'] = income_data['FIPS'].astype('int64')
income_data['YR'] = income_data['YR'].astype('str')
unemployment_data['YR'] = unemployment_data['YR'].astype('str')
unemployment_data['FIPS'] = unemployment_data['FIPS'].astype('int64')


In [126]:
unemployment_data.head(5)

,FIPS,Name,YR,Unemployment Rate (%)
0,12001,"Alachua County, FL",2014,5.4
1,12003,"Baker County, FL",2014,6.6
2,12005,"Bay County, FL",2014,6.5
3,12007,"Bradford County, FL",2014,5.8
4,12009,"Brevard County, FL",2014,7.2


In [127]:
poverty_data=poverty_data.drop(columns=['Name', 'RUC Code'])
income_data=income_data.drop(columns='Name')
unemployment_data=unemployment_data.drop(columns='Name')

In [138]:

# Merge all the datasets on 'FIPS' and 'YR'
merged_data = (
    education_data
    .merge(poverty_data, on=['FIPS', 'YR'], how='outer')  # Adjust 'how' if necessary
    .merge(income_data, on=['FIPS', 'YR'], how='outer')
    .merge(unemployment_data, on=['FIPS', 'YR'], how='outer')
)

In [142]:
merged_data.to_parquet("data/USDA_education_poverty_unemployment_income.parquet", index=False)
